In [2]:
## Standard Imports ##
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import numpy as np
import scipy as sp

import time as timeit
import datetime
import sys,os

## Kalman Filter Imports ##
from filterpy.kalman import ExtendedKalmanFilter

## Extended Kalman Filter
In this file, I develop an extended Kalman filter (EKF) solution to the identification of a single-degree-of-freedom oscillator (SDOF), in which the nonlinear switch state is turned ON, essentially developing a Bouc-Wen SDOF system. The linear stiffness contribution is turned off for this part of the example, as part of the switch state for this system. 

No hyperparameters need be selected for this inference algorithm. Typical hyperparameters in an experimental scenario include the process noise covariance $Q$ and the process noise covariance $R$. These terms are assumed known for this problem. However, as the dynamical equation for the Bouc-Wen component of the system include discontinuous terms (absolute values) a smoothed approximation is applied such that the Jacobian of the transition function can be determined. See the __Generate_Input.ipynb__ file for more details. 

This example runs 50 inference trials using varied prior information on the parameters, simulating different assertions an experimentalist might make in a practical identification scenario. Outputs from this model include:
1. The mean and standard deviation of the state and log(parameters) over the inference period. 
2. The mean of the state and parameters over the inference period. 
3. The mode of the state and parameters over the inference period. 
4. The computational model response built from the inferred parameters with respect to the input signal used for inference.
5. The runtime for each inference trial. 

The EKF implementation expressed herein is drawn from the python library FilterPy<sup>1</sup>. Some small modifications were made to adapt the library to this problem, and are shown in detail in the code below. 

__Developed by__: Alana Lund (Purdue University) \
__Last Updated__: 13 Sept. 2021 \
__License__: AGPL-3.0

### References
<sup>1</sup> R. Labbe, FilterPy v1.4.5, (2021). [https://github.com/rlabbe/filterpy](https://github.com/rlabbe/filterpy).

## Load Experimental Data

In [3]:
## Assert File Names
inFile = '../04-Data/Bouc-Wen/inferenceInput'
outFile = '../04-Data/Bouc-Wen/outputEKF'

infData = np.load(inFile + '.npz')

dt = infData['dt']                 # time step [sec]
time = infData['time']             # time history [sec]
inpAcc = infData['inpAcc']         # observations on input acceleration [m/sec^2]
states = infData['statesPNoise']   # states (for post-inference validation) [m,m/sec,m]
respAcc = infData['accPMNoise']    # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']             # process noise contributions, independent std. dev. per state [m,m/sec,m]
R = infData['Rfactor']             # measurement noise contribution [m/sec^2]
m = infData['m']                   # mass [kg]
ics = infData['ics']               # true initial conditions of the system [m, m/sec, m]
par = infData['par']               # true parameters of the system [xi (-), wn (rad/sec), beta [1/m^2], n [-], gamma [1/m^2]] 

### Lay Out Problem Dimensionality ###
nInf = 8                     # Number of inferred variables [-]
nState = states.shape[0]     # Number of states [-]
nPar = nInf - nState         # Number of parameters [-]
samps = len(time)            # Number of system measurements [-]

## Adapting FilterPy to Our Case
FilterPy includes built-in classes that help generalize the operations performed on the transmission and emission models through the EKF algorithm. However, this library assumes that the transition function is either linear or that the state transition can be well captured through the Taylor series expansion on the transition. As the Taylor series is insufficient to describe the transition dynamics in the $\alpha=1$ case, we use the full nonlinear transition function for the prediction step.  

Adjustments to the base class to expand the implementation for a nonlinear transition function are made below. Issues with the assumed system dimensionality are resolved through the careful definition of the transition and observation functions, as shown in the subsequent code segment. 

In [3]:
class myExtendedKalmanFilter(ExtendedKalmanFilter):
    """
    Here I redefine a single function called "predict_x" in the filterPy class 
    "ExtendedKalmanFilter" to specify the precise nonlinear transition function
    that is being used for prediction. This is done to compensate for a highly
    nonlinear transition function that cannot be well approximated through 
    first-order linearization.  
    """
    def predict_x(self, u=0):
        """
        State transition model for a SDOF oscillator with a Bouc-Wen switch 
        state, given that alpha = 1, and therefore the Bouc-Wen component is 
        switched on.

        states = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
                and parameters to be inferred (log(xi),log(wn), log(beta), log(n),
                log(gamma)).      
        u = input excitation at current time step [m/sec^2]
        
        Sampling rate is asserted below as there is no option in the 
        current setup to have them as inputs to the function.
        """
        dt = 1/256   # sampling rate [sec]

        state = np.squeeze(self.x.T)
        par = np.exp(state[3:]) 

        x1dot = state[0] + dt*state[1]
        x2dot = state[1] + dt*(-u - (2*par[0]*par[1])*state[1] - np.square(par[1])*state[2])
        x3dot = state[2] + dt*(state[1] - par[2]*np.absolute(state[1])*
                    np.power(np.absolute(state[2]), par[3]-1)*state[2] 
                    - par[4]*state[1]*np.power(np.absolute(state[2]), par[3]))

        self.x = np.concatenate((np.stack((x1dot, x2dot, x3dot), axis=0)
                               , state[3:]), axis=0)[:,None]
        return

### Define Transition Functions

In [4]:
def fx(x, dt, exc=None):
    """
    State transition model for a SDOF oscillator with a Bouc-Wen switch 
    state, given that alpha = 1, and therefore the Bouc-Wen component is 
    switched on.
      
    x = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
        and parameters to be inferred (log(xi),log(wn), log(beta), log(n),
        log(gamma)).
    dt = sampling rate [sec]
    exc = input excitation at current time step [m/sec^2]
    """
    if exc is None:
        exc = np.zeros(x[1].shape)
      
    par = np.exp(x[3:]) 
            
    x1dot = x[0] + dt*x[1]
    x2dot = x[1] + dt*(-exc - (2*par[0]*par[1])*x[1] - np.square(par[1])*x[2])
    x3dot = x[2] + dt*(x[1] - par[2]*np.absolute(x[1])*
                np.power(np.absolute(x[2]), par[3]-1)*x[2] 
                - par[4]*x[1]*np.power(np.absolute(x[2]), par[3]))

    return np.concatenate((np.stack((x1dot, x2dot, x3dot), axis=0)
                           , x[3:]), axis=0)

def fJacob(x, dt, exc=None):
    """
    Calculates the Jacobian of the state transition function for a SDOF
    oscillator with a Bouc-Wen switch state, given that alpha = 1, and
    therefore the Bouc-Wen component is switched n. 
      
    x = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
        and parameters to be inferred (log(xi),log(wn), log(beta), log(n),
        log(gamma)).
    dt = sampling rate [sec]
    exc = input excitation at current time step [m/sec^2]
    """
    x = np.squeeze(x.T)   # Dimensionality matching for FilterPy
   
    if exc is None:
        exc = np.zeros(x[1].shape)
        
    ## Derivatives with Respect to Transition on State x1 ##
    d1x1 = -(2*np.exp(x[3])*np.exp(x[4]))
    d1x2 = - np.square(np.exp(x[4]))
    d1p0 = -(2*np.exp(x[3])*np.exp(x[4]))*x[1]
    d1p1 = -(2*np.exp(x[3])*np.exp(x[4]))*x[1] - 2*np.square(np.exp(x[4]))*x[2]
    
    ## Derivatives with Respect to Transition on State x2 ##
    rho = 20   # smoothing constant for approximating abs(.) with tanh(.)
    alpha = np.tanh(rho*x[2])*x[2]
    eta = np.tanh(rho*x[1]*x[2])
    dalpha_dr = rho*x[2]*np.square(1/np.cosh(rho*x[2])) + np.tanh(rho*x[2])
    deta_dr = rho*x[1]*np.square(1/np.cosh(rho*x[1]*x[2]))
    deta_dx = rho*x[2]*np.square(1/np.cosh(rho*x[1]*x[2]))
    
    d2x1 = (1 - np.exp(x[7])*np.power(alpha,np.exp(x[6])) 
                    - np.exp(x[5])*np.power(alpha,np.exp(x[6]))*eta 
                    - np.exp(x[5])*x[1]*np.power(alpha,np.exp(x[6]))*deta_dx)
    d2x2 = (-np.exp(x[7])*x[1]*np.exp(x[6])*np.power(alpha,np.exp(x[6])-1)*dalpha_dr 
            - np.exp(x[5])*x[1]*(np.exp(x[6])*np.power(alpha,np.exp(x[6])-1)*dalpha_dr*eta 
                                 + np.power(alpha,np.exp(x[6]))*deta_dr))
    d2p2 = -np.exp(x[5])*x[1]*np.power(alpha,np.exp(x[6]))*eta
    d2p3 = (-(x[1]*np.exp(x[7])+x[1]*np.exp(x[5])*eta)
                *(np.exp(x[6])*np.power(alpha, np.exp(x[6]))*np.log(alpha)))
    d2p4 = -np.exp(x[7])*x[1]*np.power(alpha,np.exp(x[6]))
    
    ## Build State Transition Jacobian ##
    F = np.array([[0,1,0,0,0,0,0,0],
                 [0, d1x1, d1x2, d1p0, d1p1, 0,0,0],
                 [0, d2x1, d2x2, 0, 0, d2p2, d2p3, d2p4], 
                 [0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0]])
    
    Phi = np.eye(8) + dt*F
    return Phi

### Define Observation Functions

In [5]:
def hx(x):
    """
    Measurement model for a SDOF oscillator with a Bouc-Wen switch 
    state, given that alpha = 2, and therefore the Bouc-Wen component is 
    switched on.
      
    x = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
        and parameters to be inferred (log(xi),log(wn), log(beta), log(n),
        log(gamma)).
    """  
    x = np.squeeze(x.T)    # Dimensionality matching for FilterPy
    par = np.exp(x[3:])
    
    resp = -(2*par[0]*par[1])*x[1] - np.square(par[1])*x[2]
    return np.eye(1)*resp  # Dimensionality matching for FilterPy

def hJacob(x):
    """
    Calculates the Jacobian of the measurement function for a SDOF
    oscillator with a Bouc-Wen switch state, given that alpha = 0, and
    therefore the Bouc-Wen component is switched off. 
      
    x = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
        and parameters to be inferred (log(xi),log(wn), log(beta), log(n),
        log(gamma)).
    """
    x = np.squeeze(x.T)    # Dimensionality matching for FilterPy
    
    dx1 = -(2*np.exp(x[3])*np.exp(x[4]))
    dx2 = - np.square(np.exp(x[4]))
    dp0 = -(2*np.exp(x[3])*np.exp(x[4]))*x[1]
    dp1 = -(2*np.exp(x[3])*np.exp(x[4]))*x[1] - 2*np.square(np.exp(x[4]))*x[2]
    
    return np.array([[0,dx1,dx2,dp0,dp1,0,0,0]])

## Run EKF

In [11]:
### Load Prior Distributions on the Parameters ###
parPriors = np.loadtxt('../04-Data/parameter_priors.txt')

### Generate Storage Over States ###
muHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred parameters for each inference trial
    # over the observation period. This is what the EKF directly
    # outputs
stdHist = np.zeros((parPriors.shape[0],nInf, samps))
    # standard deviation of the inferred parameters for each 
    # inference trial over the observation period. This is 
    # what the EKF directly outputs
meanHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modeHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mode of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted.
modStates = np.zeros((parPriors.shape[0],nInf, samps))
    # Response history of the inferred system given the input
    # excitation. Essentially, we're remodeling the behavior of 
    # the system given our selections on point estimates of the 
    # parameters from the posterior. 
runTime = np.zeros(parPriors.shape[0])
    # Computational time for each inference trial. 

## For Each Inference Trial... ##
for j in range(parPriors.shape[0]):
    ### Create an Instance of the EKF Class ##
    kf = myExtendedKalmanFilter(dim_x=nInf, dim_z=1, dim_u=1)
        # Initialize UKF solver class with 8 states (3 dynamic, 5 parameters, 1 input)
    
    ### Initialize EKF Priors ###
    mu0 = np.concatenate((10**(-10)*np.ones((nState,)), parPriors[j,::2]))[:,None]
        # We start the states at not exactly 0 to avoid computational difficulties                                
    P0 = np.diag(np.square(np.concatenate((np.array([0.25, 0.25, 0.25])
                                 ,parPriors[j,1::2]))))
    
    kf.x = mu0    # Prior mean on the states and parameters
    kf.P = P0     # Prior covariance on the states and parameters
    kf.Q = np.diag(np.concatenate((np.square(Q), 10**(-8)*np.ones(nPar)))) 
        # Adding a little jitter on the parameter transition density is
        # standard practice in the joint state-parameter estimation problem. 
        # Doing so avoids singularity in the covariance on the transition dynamics
    kf.R = np.eye(1)*R**2

    ## Store Initial Values ##
    muHist[j,:,0] = np.squeeze(kf.x.T) 
    stdHist[j,:,0] = np.sqrt(np.diag(kf.P))
    meanHist[j,:,0] = np.concatenate((np.squeeze(kf.x.T)[0:nState], np.exp(np.squeeze(kf.x.T)[nState:] 
                                        + np.square(stdHist[j,nState:,0])/2)))
    modeHist[j,:,0] = np.concatenate((np.squeeze(kf.x.T)[0:nState], np.exp(np.squeeze(kf.x.T)[nState:] 
                                        - np.square(stdHist[j,nState:,0]))))

    ### Run EKF Over Data ###
    t0 = timeit.time()
    try: 
        for i in range(1,samps):
            ## Predictor ##
            kf.F = fJacob(kf.x, dt, exc=inpAcc[i-1])           
            kf.predict(u=inpAcc[i-1])
            ## Corrector ##
            kf.update(respAcc[i], hJacob, hx)

            ## Store Results ##
            muHist[j,:,i] = np.squeeze(kf.x.T) 
            stdHist[j,:,i] = np.sqrt(np.diag(kf.P))
            meanHist[j,:,i] = np.concatenate((np.squeeze(kf.x.T)[0:nState], 
                                    np.exp(np.squeeze(kf.x.T)[nState:] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((np.squeeze(kf.x.T)[0:nState], np.exp(np.squeeze(kf.x.T)[nState:] 
                                    - np.square(stdHist[j,nState:,i]))))

    except:
        print('\nERROR: Numerical Instability')
        ## Store Results ##
        for i in range(1,samps):
            muHist[j,:,i] = np.array([0,0,0, -0.1054, 2.3, 3.4, 1.9, 3.4])
            stdHist[j,:,i] = 0.01*np.ones(8)
            meanHist[j,:,i] = np.concatenate((muHist[j,:nState,i], 
                                    np.exp(muHist[j,nState:,i] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((muHist[j,:nState,i], np.exp(muHist[j,nState:,i] 
                                    - np.square(stdHist[j,nState:,i]))))
        
    ### Rerun Model with Point Estimates (Mode) of Inferred Parameters ###
    modStates[j,:,0] = np.concatenate((np.zeros((3,)), np.log(modeHist[j,3:,-1])))
    for i in range(1,samps):
        modStates[j,:,i] = fx(modStates[j,:,i-1], dt, exc=inpAcc[i-1])   
    
    tf = timeit.time()
    runTime[j] = tf-t0

    ## Print Results Summary ##
    print('\nIteration %d' %(j) )
    print('Computation Time = %.2f seconds' %((tf-t0)))
    print('Mode of Final Parameter Distributions = \n\txi = %.4f\n\twn = %.4f\n\tbeta = %.4f\n\tn = %.4f\n\tgamma = %.4f'
          %(modeHist[j,3,-1],modeHist[j,4,-1],modeHist[j,5,-1],modeHist[j,6,-1],modeHist[j,7,-1]))

### Save Output ###
np.savez(outFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, 
         modeHist=modeHist, modStates=modStates, runTime = runTime)


Iteration 0
Computation Time = 0.58 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1034
	wn = 2.8975
	beta = 24.2349
	n = 5.8670
	gamma = 17.5591

Iteration 1
Computation Time = 0.56 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0942
	wn = 2.8987
	beta = 12.3726
	n = 4.6631
	gamma = 4.9002

Iteration 2
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1085
	wn = 2.9057
	beta = 2.2009
	n = 3.9277
	gamma = 7.0839

Iteration 3
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0619
	wn = 2.9610
	beta = 10.0682
	n = 4.2061
	gamma = 5.2150

Iteration 4
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0499
	wn = 3.0185
	beta = 1.8892
	n = 2.0004
	gamma = 1.3353


C:\Users\lunda\Python\envs\pfTrial\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in power
C:\Users\lunda\Python\envs\pfTrial\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars



Iteration 5
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0933
	wn = 17.4479
	beta = 0.0256
	n = 0.0087
	gamma = 1.0316

Iteration 6
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0617
	wn = 2.9758
	beta = 2.7766
	n = 2.6448
	gamma = 1.7602

Iteration 7
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1015
	wn = 2.9211
	beta = 4.7326
	n = 4.2475
	gamma = 9.5682

Iteration 8
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1137
	wn = 2.8731
	beta = 2.6572
	n = 3.4266
	gamma = 3.0469

Iteration 9
Computation Time = 0.57 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1006
	wn = 2.9678
	beta = 2.4519
	n = 3.3543
	gamma = 6.4150

Iteration 10
Computation Time = 0.59 seconds
Mode of Final Parameter Distributions = 
	xi = 0.1089
	wn = 2.9253
	beta = 1.9202
	n = 3.1960
	gamma = 3.9897

Iteration 11
Computation Time = 0.58 seconds
Mode of Final Paramete

## Predictive Capacity of the Inferred Models
The goal of this section is to develop a prediction of the response behavior of the system to a secondary event, given the models which have been inferred from the primary excitation. 

### Load Inference Data
This becomes an optional start point in the code. If the data for the EKF has already been generated, it can simply be loaded in for the predictive analysis instead of rerunning the previous block of code. 

In [6]:
outData = np.load(outFile + '.npz')

muHist = outData['muHist']         # inference history of untransformed state/par means
stdHist = outData['stdHist']       # inference history of state/par standard deviations
meanHist = outData['meanHist']     # inference history of transformed state/par means
modeHist = outData['modeHist']     # inference history of transformed state/par modes
modStates = outData['modStates']   # states that have been remodeled based on the final modes of the parameters

### Load Secondary Input Excitation

In [7]:
predInFile = '../04-Data/Bouc-Wen/predInp_BLWN'
predOutFile = '../04-Data/Bouc-Wen/predOutEKF'

infData = np.load(predInFile + '.npz')

dt = infData['dt']                            # time step [sec]
time = infData['time']                        # time history [sec]
predBase = infData['predInp']                 # observations on input acceleration [m/sec^2]
predStatesTrue = infData['predStatesPNoise']  # states (for post-prediction validation) [m,m/sec,m]
predRespTrue = infData['predAccPMNoise']      # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']                        # process noise contributions, independent std. dev. per state [m,m/sec,m]
R = infData['Rfactor']                        # measurement noise contribution [m/sec^2]
m = infData['m']                              # mass [kg]
ics = infData['ics']                          # true initial conditions of the system [m, m/sec, m]
par = infData['par']                          # true parameters of the system [xi (-), wn (rad/sec), beta [1/m^2], n [-], gamma [1/m^2]] 

### Generate Predictive Distribution on the States over Secondary Input

In [10]:
### Set Constants for Predictive Sampling ###
nPriors = muHist.shape[0]    # Number of inference trials [-]
nSamps = 500                 # Number of samples on the inference posterior [-]
seeds = [8192,3245]          # seeds for random number generator

### Generate Storage Over Predicted States ###
totalSamps = np.zeros((nSamps*nPriors, nState, len(time)))
    # Predicted states based on simulations results for all posterior
    # samples from all inference trials
meanPred = np.zeros((nPriors, nState, len(time)))
    # Mean of the predicted states for each inference trial.
stdPred = np.zeros((nPriors,nState, len(time)))
    # Standard deviation of the predicted states for each inference trial. 

### Run Predictive Trials on All Candidate Models ###
print('Predictive Distribution from Inferred Results')
for j in range(nPriors):
    print('Case %d'%(j))
    ## Random Samples on the States and Parameters, based on Inferred Posterior ##
    np.random.seed(seeds[0]+j)
    rSamp = np.random.multivariate_normal(np.zeros(nInf), np.eye(nInf), nSamps)
    predSamps = muHist[j,:,-1] + stdHist[j,:,-1]*rSamp

    ## Random Samples on the Transition Noise ##
    np.random.seed(seeds[1]+j)
    noise = Q.reshape(-1,1)*np.random.multivariate_normal(np.zeros(nState), np.eye(nState), 
                                                          (nSamps, len(time))).transpose((0, 2, 1))

    ## Prepare Response Storage ##
    predStates = np.zeros((nSamps, nState,len(time)))
    predStates[:,:,0] = predSamps[:,:nState]

    for i in range(nSamps):
        for tt in range(1,len(time)):
            predStates[i,:,tt] = fx(np.concatenate((predStates[i,:, tt-1], predSamps[i,nState:])), 
                                                 dt, exc = predBase[tt-1])[:nState] + noise[i,:,tt-1]
    
    ## Store Results from Predictive Sample Runs ##
    meanPred[j,:,:] = np.mean(predStates, axis = 0)
    stdPred[j,:,:] = np.sqrt(np.mean(np.square(predStates), axis=0) - np.square(meanPred[j,:,:])) 
    totalSamps[j*nSamps:(j+1)*nSamps,:,:] = predStates
    
### Remove Unstable Results from the Overall Assessment ###
# Candidate models can become unstable during inference (due to 
# computational issues such as singularities in the covariance 
# matrices) or manifest instability during predictive modeling
# due to combinations of the selected parameters which result in
# model divergence. Here we extract these cases so that they don't 
# interfere with the statistics of the main results. 
stabilityInd = np.ones(nPriors)
totalStabilityInd = np.ones(nPriors*nSamps)

print('\nIndices of Unstable Predictive Distributions:')
for i in range(nPriors):
    if (np.isnan(meanPred[i,0,-1])) or (np.absolute(meanPred[i,0,-1])>100) or (muHist[i,0,-1] == 0):
        stabilityInd[i] = 0 
        totalStabilityInd[i*nSamps:(i+1)*nSamps] = np.zeros(nSamps)
        print(i)

stableMeans = meanPred[stabilityInd != 0,:,:]
stableStds = stdPred[stabilityInd != 0,:,:]
stableSamps = totalSamps[totalStabilityInd != 0,:,:]

### Statistics on all Stable Cases ###
meanAll = np.mean(stableSamps, axis = 0)
stdAll = np.sqrt(np.mean(np.square(stableSamps), axis=0) - np.square(meanAll)) 

### Save Output ###
np.savez(predOutFile, meanPred = meanPred,stdPred=stdPred, 
         stableMeans=stableMeans, stableStds=stableStds, meanAll=meanAll, stdAll=stdAll)

Predictive Distribution from Inferred Results
Case 0
Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7
Case 8
Case 9
Case 10
Case 11
Case 12
Case 13
Case 14
Case 15
Case 16
Case 17
Case 18
Case 19
Case 20
Case 21
Case 22
Case 23
Case 24
Case 25
Case 26
Case 27
Case 28
Case 29
Case 30
Case 31
Case 32


C:\Users\lunda\Python\envs\pfTrial\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in power
C:\Users\lunda\Python\envs\pfTrial\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\lunda\Python\envs\pfTrial\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in square


Case 33
Case 34
Case 35
Case 36
Case 37
Case 38
Case 39
Case 40
Case 41
Case 42
Case 43
Case 44
Case 45
Case 46
Case 47
Case 48
Case 49

Indices of Unstable Predictive Distributions:
32
